In [ ]:
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    M2M100Tokenizer,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
import torch.nn.functional as F
from datasets import load_dataset
import re
import csv
from tqdm import tqdm

Found cached dataset csv (/home/oleksandr.yermilov/.cache/huggingface/datasets/csv/default-fb3f541b93165b8b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.27it/s]

By . Floor Blythe . Pep Guardiola will never be sacked as Bayern Munich head coach according to the Bundesliga champions' chairman. Karl-Heinz Rummenigge was questioned about whether or not he was worried that many of Bayern's German World Cup-winning stars had yet to return to pre-season training when he made the claim. German newspaper Welt am Sonntag carried an article on Sunday claiming Guardiola's side could struggle this season due to the tiring World Cup campaign. VIDEO Scroll down to watch Pep Guardiola lose it with a journalist and get soaked in beer . Forever: Bayern Munich chairman Jörn-Heinz Rummenigge says the club will 'never' sack boss Pep Guardiola.


In [ ]:
path_to_model = ""
path_to_train_dataset = ""
path_to_output = ""

In [ ]:
generation_config = dict(
    max_length=4096,
    min_length=None,
    do_sample=False,
    early_stopping=True,
    num_beams=1,
    temperature=15.0,
    top_k=None,
    top_p=None,
    length_penalty=1.0,
    num_return_sequences=3,
    max_time=None,
    num_beam_groups=1,
    output_scores=False,
)

rank = 0
model_path = path_to_model

_tokenizer = AutoTokenizer.from_pretrained(model_path)
_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
pipe = pipeline("translation", model = _model, config = generation_config, batch_size=1, tokenizer = _tokenizer, max_length = 4096)

# create a dataloader
dataset = load_dataset("csv", data_files=path_to_train_dataset)
with open(path_to_output, "w") as f:
        writer = csv.writer(f)
        for out in tqdm(pipe(KeyDataset(dataset['train'], "source"), batch_size=8, truncation="only_first")):
            writer.writerow(out[0]["translation_text"])
